### Test with TensorflowDatasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test)

Epoch 1/6
469/469 [==============================] - 2s 2ms/step - loss: 0.6227 - sparse_categorical_accuracy: 0.8306 - val_loss: 0.1992 - val_sparse_categorical_accuracy: 0.9426
Epoch 2/6
469/469 [==============================] - 1s 1ms/step - loss: 0.1794 - sparse_categorical_accuracy: 0.9480 - val_loss: 0.1396 - val_sparse_categorical_accuracy: 0.9588
Epoch 3/6
469/469 [==============================] - 1s 1ms/step - loss: 0.1230 - sparse_categorical_accuracy: 0.9661 - val_loss: 0.1076 - val_sparse_categorical_accuracy: 0.9678
Epoch 4/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0925 - sparse_categorical_accuracy: 0.9731 - val_loss: 0.0948 - val_sparse_categorical_accuracy: 0.9716
Epoch 5/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0691 - sparse_categorical_accuracy: 0.9802 - val_loss: 0.0857 - val_sparse_categorical_accuracy: 0.9754
Epoch 6/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0581 - sparse_categorical_accu

### Test with Keras Dataset load_data()

In [6]:
import tensorflow as tf

(train_x,train_y), (test_x,test_y) = tf.keras.datasets.mnist.load_data()

print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


### Customized Keras Model

In [17]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        if training:
            x = self.dropout(x, training=training)
        return self.dense2(x)

model = MyModel()
model.build((None, 16))

model.summary()

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              multiple                  68        
_________________________________________________________________
dense_7 (Dense)              multiple                  25        
_________________________________________________________________


ValueError: You tried to call `count_params` on dropout_3, but the layer isn't built. You can build it manually via: `dropout_3.build(batch_input_shape)`.

### Run Experiment

In [16]:
!python3 keras_version/exec_experiment.py --architecture "61, 24, 60,  5, 57, 55, 59, 3"

train.shape: (60000, 28, 28),(60000,)
test.shape: (10000, 28, 28),(10000,)
action: [[61, 24, 60, 5], [57, 55, 59, 3]]
cnn_drop_rate: [5, 3]
num_classes: 10
cnn_config: [[61, 24, 60, 5], [57, 55, 59, 3]]
cnn_drop_rate: [5, 3]
batch_size: 100
cnn: [61, 57]
cnn_num_filters: [24, 55]
max_pool_ksize: [60, 59]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_out_0 (Conv2D)          (None, 100, 28, 24)       2500536   
_________________________________________________________________
max_pool_0 (MaxPooling2D)    (None, 100, 28, 24)       0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 28, 24)       0         
_________________________________________________________________
conv_out_1 (Conv2D)          (None, 100, 28, 55)       4288735   
_________________________________________________________________
max_pool_1 (Ma